In [ ]:
import numpy as np
from avapi.nuscenes import nuScenesManager


nusc_data_dir = "../../data/nuScenes"
NSM = nuScenesManager(nusc_data_dir)
NSD = NSM.get_scene_dataset_by_name("scene-0103")

## Test Visualization

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes, show_lidar_bev_with_boxes


frame = 20
img = NSD.get_image(frame, "main_camera")
pc = NSD.get_lidar(frame, "main_lidar")
objects = NSD.get_objects(frame, "main_lidar")

# -- 2d camera
show_image_with_boxes(img, objects, inline=True)

# -- 2d bev
vectors = [obj.velocity_head_tail for obj in objects]
show_lidar_bev_with_boxes(pc, boxes=objects, vectors=vectors)

## Test Object Tracking

In [ ]:
from avstack.datastructs import DataContainer
from avstack.modules.perception.detections import BoxDetection
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D


# initialize a tracker
tracker = BasicBoxTracker3D(check_reference=True)

# detection noise for tracker
noise = np.array([1, 1, 1, 0.25, 0.25, 0.25]) ** 2

# run tracking
first_frame = 1
last_frame = 20
for frame in NSD.frames[first_frame:last_frame]:
    # get data from this frame
    ts = NSD.get_timestamp(frame)
    ego = NSD.get_ego(frame)
    img = NSD.get_image(frame, sensor="main_camera")
    objects = NSD.get_objects(frame, sensor="main_camera")

    # convert "objects" into "detections" (ground truth)
    detections = DataContainer(
        frame=frame,
        timestamp=ts,
        data=[
            BoxDetection(
                data=obj.box3d,
                noise=noise,
                source_identifier="objects",
                reference=obj.reference,
                obj_type=obj.obj_type,
            )
            for obj in objects
        ],
        source_identifier="objects",
    )

    # perform tracking
    tracks = tracker(
        t=ts, frame=frame, detections=detections, platform=ego.as_reference()
    )
    if frame > 1:
        show_image_with_boxes(img, tracks, inline=True, show_IDs=True)

## Test Radar Tracking

In [ ]:
from avstack.modules.tracking.tracker3d import BasicRazelTracker
from avstack.modules.perception.detections import RazelDetection


# initialize tracker
tracker_radar = BasicRazelTracker(threshold_confirmed=2, assign_radius=10)
noise_radar = np.array([1, 1, 1]) ** 2

# loop over frames
last_frame = 20
for frame in NSD.frames[:last_frame]:
    # get data
    ts = NSD.get_timestamp(frame)
    radar = NSD.get_radar(frame)
    ego = NSD.get_ego(frame)
    objects = NSD.get_objects(frame)
    img = NSD.get_image(frame, "main_camera")

    # make razel detections
    detections = DataContainer(
        frame=frame,
        timestamp=ts,
        data=[
            RazelDetection(
                data=radar.data[i, :3],
                noise=noise_radar,
                source_identifier="objects",
                reference=radar.reference,
            )
            for i in range(radar.data.shape[0])
        ],
        source_identifier="objects",
    )

    # filter only to the front view
    detections.filter(lambda det: det.xyz[0] > 0)

    # run tracker
    tracks = tracker_radar(
        t=ts, frame=frame, detections=detections, platform=radar.reference
    )

    # show images with tracks
    if frame > 1:
        show_image_with_boxes(img, tracks, inline=True)

In [ ]:
from avstack.geometry.transformations import matrix_spherical_to_cartesian


# visualize result
frame = 6
pc = NSD.get_lidar(frame=frame)
pc.data = np.zeros((0, 4))
objects = NSD.get_objects(frame=frame)
rad = NSD.get_radar(frame=frame)
det_xyz = matrix_spherical_to_cartesian(rad.data.x)

# -- vectors from detections
vectors = [obj.velocity_head_tail for obj in objects]
show_lidar_bev_with_boxes(pc, boxes=objects, vectors=vectors)

# -- vectors from tracks
# vectors = [obj.velocity_head_tail for obj in tracks]
# avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=tracks, vectors=vectors)